In [1]:
import numpy as np
import kf_recon_deep_win as kf_recon_deep
import matplotlib.pyplot as plt
from scipy.optimize import minimize

The sign of the probe angle is inferred by the port numbers in the R_PDV/Rapid-Test-Bed/Data_Package_EFP_1_SS/IMG_3660.jpg file. For ports with azimuthal angles away from the main cluster, the probe sign is chosen somewhat arbitrarily to balance the beams.

Initialize the probe angles and values of theta to perform the reconstruction:

In [3]:
probe_signs = np.array([1,1,1,-1,-1,1,-1,-1,1,1,1,1,-1,-1,1,-1])

probes = np.loadtxt(open("../Data/Probe_Angles.txt", "rb"), skiprows=1)[0:16,1]
probes = probes*np.pi/180.*probe_signs
probes = np.delete(probes,[6,9,11])

pred_ang = np.arange(10)*np.pi/9. - np.pi/2.

Read in velocity data from a csv 

For now velocity data should be formatted with 
    - each column as a length n_times vector of time observations
    - order of the columns corresponds to the order of the probe angles above

In [4]:
pdv_vel = np.loadtxt(open("../Data/PDV_traces2.txt", "rb"), skiprows=1)
times   = pdv_vel[:,0]*1.e6
dt      = times[1] - times[0]
pdv_vel = pdv_vel[:,np.arange(1,26,2)]#/1.0e6

In [5]:
# Drop beams that are azimuthally off from the others.
pdv_vel = np.delete(pdv_vel, [2,4,6,7,9],1)
probes  = np.delete(probes,[2,4,6,7,9])


In [6]:
pdv_pos = np.zeros(pdv_vel.shape)
pdv_pos.fill(np.nan)

Set values to _np.nan_ that are places where the probe did not actually have a valid measurement. 

I added a function to the module to do so in a way that seems sensible for the data I've had, but user should do so in the way that makes sense for their application.

The model will treat these as missing values. 

This typically is happening at late times for the probe, where the velocity suddenly drops to negative values.

In [7]:
kf_recon_deep.set_nan(pdv_vel)
pdv_vel /= 1.0e6
pdv_vel *= 1.0e3


Capability for beam offset in progress. For now the offset should be zero.

In [8]:
offsets = np.zeros(13)

Set up the reconstruction parameters. Will be generalize later but for now:
    - How many observations are there in time
    - What is the initial radius of the shell
    - What is the horizontal offset from (0,0) that the probes intersect with the y=0
          - Current below has beams equally spaced over a 0.005 m interval and that grouping is randomly 
              shifted off of center by draw from a random normal with standard dev of 0.01
    - Measured velocities
    - What is the size of the timestep

In [9]:
def fit_ll(params):
    tmp = kf_recon_deep.Reconstruction(n_times      = pdv_vel.shape[0], 
                               init_r       = 0.083997*1.0e3,
                               probe_angles = probes,
                               probe_offset = offsets,
                               pred_angles  = np.arange(2)*np.pi/1. - np.pi/2., 
                               probe_vel    = pdv_vel,
                               probe_pos    = pdv_pos,
                               times        = times,
                               dt           = dt)
    print(params)
    tmp.beta      = 10**params[0]
    tmp.kappa     = 10**params[1]
    tmp.sigsq_vel = 10**params[2]
    tmp.lam       = 10**params[3]
    tmp.lam2      = 10**params[4]
    tmp.lin_scale = 1.0 
    tmp.con_scale = 100.
    tmp.fit_model()
    print(-tmp.l_like)
    return -tmp.l_like

In [10]:
parm0 = np.array([np.log10(np.pi/2.0), 
                  np.log10(5.), 
                  np.log10(0.05), 
                  np.log10(np.sqrt(3)/1.), 
                  np.log10(np.sqrt(3)/10.)])
res = minimize(fit_ll, parm0, method='nelder-mead', options={'fatol': 1e-1, 'disp': True})

/Users/mikegros/Projects/PDV_Reconstruction/py_recon/kf_recon_deep_win.py:149: RuntimeWarning: invalid value encountered in greater
  first_big_vel = np.where( self.probe_vel_orig[:, ii] > 0.100)[0][0]
/Users/mikegros/Projects/PDV_Reconstruction/py_recon/kf_recon_deep_win.py:150: RuntimeWarning: invalid value encountered in less
  first_peak    = np.where( np.diff( self.probe_vel_orig[first_big_vel:, ii] ) < 0)[0][0] + 1


[ 0.19611988  0.69897    -1.30103     0.23856063 -0.76143937]
-19388.348008704812
[ 0.20592587  0.69897    -1.30103     0.23856063 -0.76143937]
-19390.513054367762
[ 0.19611988  0.7339185  -1.30103     0.23856063 -0.76143937]
-19379.587739141207
[ 0.19611988  0.69897    -1.3660815   0.23856063 -0.76143937]
-21941.362542250183
[ 0.19611988  0.69897    -1.30103     0.25048866 -0.76143937]
-19385.430057690348
[ 0.19611988  0.69897    -1.30103     0.23856063 -0.79951134]
-19401.55788789598
[ 0.20004227  0.6640215  -1.3270506   0.24333184 -0.77666816]
-20423.457738587593
[ 0.20161123  0.6849906  -1.33745884  0.22854108 -0.78275968]
-20832.81279892937
[ 0.20380778  0.67939884 -1.35203037  0.23646129 -0.7912878 ]
-21407.540351641983
[ 0.19315454  0.67157038 -1.37243052  0.23562156 -0.80322717]
-22211.94203608297
[ 0.18676888  0.65787057 -1.40813078  0.23415203 -0.82412106]
-23621.425661289475
[ 0.19922014  0.65513061 -1.41527084  0.23385812 -0.77499908]
-23887.52905688305
[ 0.20077027  0.6332

-109805.34888339444
[ 0.13684034 -0.39331333 -4.23357913  0.42002533 -0.01501448]
-112389.22420250202
[ 0.1397705  -0.40227326 -4.31842367  0.39315922 -0.150842  ]
-110592.02892192244
[ 0.14633448 -0.43896111 -4.40557418  0.43022424  0.04695734]
-112417.52452953823
[ 0.15763089 -0.49420992 -4.55775371  0.48421116  0.3056282 ]
-116472.55416161065
[ 0.15498248 -0.44224213 -4.38678787  0.4814565   0.29419793]
-116703.71630384379
[ 0.16885014 -0.4766252  -4.45764504  0.55621954  0.65393037]
-120669.73031900483
[ 0.15743398 -0.5258513  -4.59021215  0.54193047  0.49434162]
-120090.87516636052
[ 0.17026063 -0.51063673 -4.5536537   0.574964    0.72277711]
-121828.04903924899
[ 0.1884161  -0.56281886 -4.67578467  0.67081886  1.18794547]
-120046.2666946057
[ 0.1766359  -0.55798134 -4.63871382  0.63778098  1.01550713]
-122739.2959358106
[ 0.1950686  -0.63583538 -4.7988589   0.76009186  1.5986817 ]
-106261.55441102468
[ 0.19548427 -0.63280846 -4.88561224  0.69801713  1.29188825]
-120512.9533410743

-127835.4658764557
[ 0.18263727 -0.75920748 -5.25622655  0.69102925  1.12455308]
-127891.14679386531
[ 0.18285308 -0.76044417 -5.26003893  0.69165087  1.1274382 ]
-127829.72188186519
[ 0.1827042  -0.75923146 -5.25630566  0.69113644  1.12554554]
-127829.8467567004
[ 0.18268614 -0.75871642 -5.25451539  0.69097615  1.12501592]
-127828.53142004261
[ 0.18276614 -0.75934949 -5.25697256  0.69107479  1.12517475]
-127827.83214334366
[ 0.18275726 -0.75921884 -5.25653643  0.69105887  1.12514579]
-127827.86503386966
[ 0.18275948 -0.7592515  -5.25664546  0.69106285  1.12515303]
-127827.8574258561
[ 0.18278253 -0.75901439 -5.25645624  0.69078534  1.12390085]
-127885.7260498546
[ 0.18267074 -0.75921947 -5.25626611  0.69108284  1.12504931]
-127830.371640409
[ 0.18274518 -0.75982582 -5.25813274  0.69134006  1.12599564]
-127830.37391545795
[ 0.18266171 -0.75896195 -5.25537097  0.6910027   1.1247845 ]
-127889.9349287435
[ 0.18270171 -0.75927848 -5.25659955  0.69105202  1.12486391]
-127889.60725100074
[ 0

-128089.45078656191
[ 0.17043542 -0.74873195 -5.18080231  0.69333833  1.1077834 ]
-127781.34133297352
[ 0.17079419 -0.7484246  -5.18100876  0.69310475  1.10804882]
-127774.26630774155
[ 0.17122059 -0.74652956 -5.1760336   0.69253787  1.10874822]
-128069.96024900726
[ 0.1700725  -0.74698869 -5.17429675  0.69291418  1.10623306]
-127770.54250985454
[ 0.17139633 -0.74707518 -5.17830669  0.69263281  1.10921705]
-128072.48511366129
[ 0.1687666  -0.74432489 -5.16073525  0.69293775  1.10534155]
-128086.19941798998
[ 0.16920872 -0.74591195 -5.16753053  0.69310882  1.10579498]
-127769.97781686051
[ 0.17071762 -0.74637516 -5.17390783  0.69268061  1.10800991]
-128074.07118328277
[ 0.16876151 -0.74508632 -5.1634979   0.69303299  1.1048433 ]
-127768.21857109462
[ 0.17073762 -0.74657797 -5.17460449  0.69273286  1.10812361]
-128075.96327403301
[ 0.16944821 -0.74586746 -5.16817542  0.6930061   1.10609592]
-128088.97936483187
[ 0.16891244 -0.74538045 -5.16490713  0.69306314  1.10517114]
-127769.05612713

-132823.4995966686
[-0.14557531 -1.12191415 -5.17385558  0.96888966  1.12664542]
-133965.7499079573
[-0.19739162 -1.15184534 -5.07489679  1.00471833  1.11468338]
-133205.5959901113
[-0.12790937 -1.11896208 -5.23194555  0.95850192  1.13361879]
-134377.36642728827
[-0.15200507 -1.1883335  -5.3616348   0.99127495  1.153585  ]
-135846.4495822771
[-0.21556084 -1.22086585 -5.22453989  1.0345825   1.13874206]
-135161.90775419242
[-0.20517809 -1.23825279 -5.31990585  1.03381693  1.15030132]
-136098.39812177868
[-0.25977774 -1.34386082 -5.45004841  1.09311742  1.17149543]
-137785.05074301997
[-0.34576488 -1.43046288 -5.40010078  1.16391638  1.17018029]
-138098.71211789214
[-0.49746764 -1.65556183 -5.54320648  1.30931619  1.19933033]
-140480.11233526425
[-0.31076302 -1.46036912 -5.62641727  1.15415395  1.20123592]
-139434.08808289174
[-0.42865442 -1.6256823  -5.70848315  1.26408835  1.21911008]
-141161.97871340744
[-0.57019397 -1.87756637 -5.97579693  1.4116877   1.2653424 ]
-143580.3334720246
[

-145557.6971109061
[-0.7023518  -2.14047229 -6.33961502  1.55138111  1.30242388]
-145674.0451022137
[-0.69915945 -2.11813335 -6.28872503  1.54139733  1.28856217]
-145770.4864774339
[-0.69860734 -2.1345318  -6.3399727   1.54648756  1.29791487]
-145807.50784927412
[-0.69357359 -2.11667484 -6.3111993   1.53658806  1.28612903]
-145966.6767567831
[-0.70282866 -2.13228124 -6.32482099  1.54626092  1.28968265]
-145968.5535162271
[-0.70357051 -2.12014822 -6.29539187  1.5404098   1.27589169]
-146296.20551928203
[-0.68564193 -2.09710242 -6.29006376  1.52417523  1.27370909]
-146270.52465234822
[-0.70026419 -2.08925355 -6.21665838  1.52813774  1.25878967]
-146403.83192223436
[-0.70257786 -2.05603563 -6.12044766  1.51562411  1.23059291]
-146574.42055370353
[-0.68600224 -2.04293932 -6.14336547  1.5025914   1.23131376]
-146648.29714439248
[-0.67509981 -1.98425975 -6.02556543  1.4735439   1.19165054]
-146891.20181280703
[-0.68502603 -2.03155499 -6.12834218  1.49473911  1.21462714]
-146986.53835426137
[

-148895.92376209112
[-0.67598157 -2.07561727 -6.42373011  1.47440197  1.15409761]
-148882.8793146889
[-0.67960248 -2.08069357 -6.42776032  1.47742042  1.15351625]
-148897.65154164945
[-0.68086583 -2.08334443 -6.43043376  1.47908282  1.15477831]
-148896.14558588003
[-0.68338502 -2.0854794  -6.42917087  1.48069791  1.15353941]
-148886.04047010595
[-0.67806214 -2.079005   -6.42731127  1.47632466  1.15412155]
-148898.53810484335
[-0.67942315 -2.08100608 -6.42830159  1.47767699  1.15452162]
-148895.20539150783
[-0.67957431 -2.08074555 -6.42769247  1.47750015  1.15379934]
-148894.18940549195
[-0.67870104 -2.08056819 -6.43018083  1.47704142  1.15430186]
-148895.68942379815
[-0.67947121 -2.08141248 -6.43040535  1.4775893   1.15399921]
-148898.5288718503
[-0.68010289 -2.08273791 -6.43174207  1.47842049  1.15463024]
-148897.56952128606
[-0.67954458 -2.08055948 -6.4269861   1.4774838   1.15389219]
-148900.19943865787
[-0.67948232 -2.08139505 -6.4302699   1.47759969  1.15402037]
-148897.6683596655

-148901.94480257
[-0.67950276 -2.08295078 -6.434916    1.47810912  1.15485525]
-148907.5706865803
[-0.67948273 -2.08296831 -6.43504787  1.4781034   1.15487233]
-148901.9420077574
[-0.67950432 -2.08295576 -6.43492459  1.47811169  1.15485767]
-148907.57498674313
[-0.679477   -2.08297617 -6.43509295  1.47810296  1.15487962]
-148901.94075598943
[-0.67950441 -2.08295441 -6.43492052  1.47811123  1.15485663]
-148907.57830536217
Optimization terminated successfully.
         Current function value: -148907.626465
         Iterations: 417
         Function evaluations: 712


In [12]:
res["x"]

array([-0.67949618, -2.08296688, -6.43499212,  1.47811026,  1.15486591])

In [40]:
print('Spatial Correlation Length: ', (10**res['x'][0])/np.pi*180)
print('Process SD: ', np.sqrt(10**res['x'][1]) )
print('Error Standard Deviation: ', np.sqrt(10**res['x'][2]) )
print('Temporal Correlation Length: ', np.sqrt(3)/10**res['x'][3] )
print('Temporal Correlation Length: ', np.sqrt(3)/10**res['x'][4] )


Spatial Correlation Length:  11.984680185308637
Process SD:  0.09089009543268504
Error Standard Deviation:  0.0006060436972451686
Temporal Correlation Length:  0.05760369821833496
Temporal Correlation Length:  0.12125362161709696
